## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-02-16-52-15 +0000,nyt,"Tesla Car Sales Dropped 9% in 2025, Falling Be...",https://www.nytimes.com/2026/01/02/business/te...
1,2026-01-02-16-49-51 +0000,bbc,Charity provides hot lunches as temperatures drop,https://www.bbc.com/news/articles/c1lz4143gdro...
2,2026-01-02-16-48-11 +0000,bbc,Anthony Joshua's driver charged over Nigeria c...,https://www.bbc.com/news/articles/c5y2jrdnnqyo...
3,2026-01-02-16-47-31 +0000,nypost,"Megyn Kelly declares CBS News, legacy media ‘d...",https://nypost.com/2026/01/02/media/megyn-kell...
4,2026-01-02-16-45-51 +0000,nyt,Live Updates: Swiss Authorities Say Sparklers ...,https://www.nytimes.com/live/2026/01/02/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2340/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
41,new,30
42,year,28
165,trump,21
43,fire,20
36,swiss,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
166,2026-01-01-22-55-18 +0000,wapo,"About 40 killed, 115 injured in New Year’s bar...",https://www.washingtonpost.com/world/2026/01/0...,125
110,2026-01-02-08-05-41 +0000,nypost,Swiss face painful task of identifying victims...,https://nypost.com/2026/01/02/world-news/swiss...,124
156,2026-01-02-01-34-44 +0000,nyt,Fire in Swiss Alps Leaves Dozens of New Year’s...,https://www.nytimes.com/2026/01/01/world/europ...,108
4,2026-01-02-16-45-51 +0000,nyt,Live Updates: Swiss Authorities Say Sparklers ...,https://www.nytimes.com/live/2026/01/02/world/...,106
131,2026-01-02-04-30-00 +0000,wsj,President Trump veered between business and pl...,https://www.wsj.com/politics/policy/golf-peace...,104


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
166,125,2026-01-01-22-55-18 +0000,wapo,"About 40 killed, 115 injured in New Year’s bar...",https://www.washingtonpost.com/world/2026/01/0...
105,51,2026-01-02-08-56-35 +0000,nypost,Trump vows US ‘locked and loaded’ to help Iran...,https://nypost.com/2026/01/02/world-news/trump...
188,35,2026-01-01-19-41-07 +0000,nypost,Swiss nightclub where 47 people died advertise...,https://nypost.com/2026/01/01/world-news/swiss...
124,35,2026-01-02-05-51-10 +0000,nypost,Kim Jong Un’s daughter visits family mausoleum...,https://nypost.com/2026/01/02/world-news/kim-j...
31,28,2026-01-02-15-03-38 +0000,bbc,China's BYD overtakes Tesla as world's top EV ...,https://www.bbc.com/news/articles/cj9rjwpvmpzo...
60,28,2026-01-02-12-27-57 +0000,nypost,Mayor Mamdani blasted for revoking city polici...,https://nypost.com/2026/01/02/us-news/nyc-mayo...
168,24,2026-01-01-22-24-25 +0000,nypost,Zelensky says peace deal is ‘90% ready’ in New...,https://nypost.com/2026/01/01/world-news/zelen...
160,24,2026-01-02-00-31-50 +0000,nypost,"Terrifying video, cockpit audio captures momen...",https://nypost.com/2026/01/01/us-news/terrifyi...
43,23,2026-01-02-14-00-05 +0000,nyt,Big Changes Are Coming to Student Loans. Here’...,https://www.nytimes.com/2026/01/02/business/st...
191,23,2026-01-01-19-02-25 +0000,nypost,Iranian protesters bring regime to halt as fur...,https://nypost.com/2026/01/01/world-news/irani...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
